In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb dill openpyxl torchmetrics
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions, sequential_generate_solutions
from temp.data.problem import setcover, lot_sizing
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst, with_lic


In [4]:
PRE_TRAIN_DIR = "temp/pre_train_tiny"
TRAIN_DIR = "temp/train_tiny_combined"
VALID_DIR = "temp/valid_tiny"

In [5]:
import json
with open("gb.lic") as f:
    lic = json.load(f)

# parallel_generate_problem(lot_sizing, PRE_TRAIN_DIR, 1000, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, TRAIN_DIR, 5000, 10)
# parallel_generate_solutions(TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, VALID_DIR, 1000, 10)
# parallel_generate_solutions(VALID_DIR, 5, lic=lic)

In [6]:
# import os
# import shutil

# # Define the source and destination folders
# src_folder = "temp/pre_train_tiny"
# dest_folder = "temp/pre_train_tiny_backup"

# # Create the destination folder if it doesn't exist
# os.makedirs(dest_folder, exist_ok=True)

# # List all files in the source folder
# files = os.listdir(src_folder)

# # First, process all .lp files: if a corresponding .npz file exists, move both to "done"
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".lp"):
#         lp_path = os.path.join(src_folder, file_name)
#         base_name = os.path.splitext(file_name)[0]
#         npz_file = base_name + ".npz"
#         npz_path = os.path.join(src_folder, npz_file)
        
#         # If the corresponding .npz exists, move both files
#         if os.path.exists(npz_path):
#             shutil.move(lp_path, os.path.join(dest_folder, file_name))
#             shutil.move(npz_path, os.path.join(dest_folder, npz_file))
#             print(f"Moved pair: {file_name} and {npz_file}")

# # Next, remove any .npz files that are left without a corresponding .lp file.
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".npz"):
#         base_name = os.path.splitext(file_name)[0]
#         lp_file = base_name + ".lp"
#         lp_path = os.path.join(src_folder, lp_file)
#         npz_path = os.path.join(src_folder, file_name)
        
#         if not os.path.exists(lp_path):
#             os.remove(npz_path)
#             print(f"Removed orphan npz: {file_name}")

In [7]:
# m = lot_sizing()
# model = m

# # model.setParam("OutputFlag", 0)
# model.setParam("PoolSolutions", 10)
# model.setParam("PoolSearchMode", 2)
# model = with_lic(model)
# model.optimize()

# vs = model.getVars()
# obj_val_and_sols = []

# for i in range(model.SolCount):
#     # TODO: setting solution number actually takes long time try to optimize
#     # TODO: add function to round by tolerance
#     model.params.SolutionNumber = i
#     obj_val = model.PoolObjVal
#     s = [obj_val] + [v.Xn for v in vs]
#     obj_val_and_sols.append(s)
# model.dispose()

# info = ModelInfo.from_model(m)
# info.var_info.sols = np.array(obj_val_and_sols)

In [8]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [9]:
# import os
# import pickle
# from temp.data.aug import augment_info, get_constraint_violations, get_lhs_matrix

# for p in os.listdir(os.path.join(PRE_TRAIN_DIR, "processed")):
#     if not p.endswith("pkl"):
#         continue
#     with open(os.path.join(PRE_TRAIN_DIR, "processed", p), "rb") as f:
#         info = pickle.load(f)

#     lhs = get_lhs_matrix(info.var_info.n, info.con_info)
#     rhs = np.array(info.con_info.rhs)
#     ops = np.array(info.con_info.types)
#     violations = get_constraint_violations(lhs, info.var_info.sols[0, 1:], rhs, ops)
#     assert not (np.abs(violations) >= 1e-5).any(), violations

    
#     # for _ in range(100):
#     #     aug_info = info.copy()
#     #     augment_info(aug_info)
#     #     lhs = get_lhs_matrix(aug_info.var_info.n, aug_info.con_info)
#     #     rhs = np.array(aug_info.con_info.rhs)
#     #     ops = np.array(aug_info.con_info.types)
#     #     violations = get_constraint_violations(lhs, aug_info.var_info.sols[0, 1:], rhs, ops)
#     #     assert not (np.abs(violations) >= 1e-5).any(), violations

In [10]:
# import gurobipy as gp

# from temp.data.info import ModelInfo
# from temp.data.problem import lot_sizing, setcover
# from temp.solver.utils import with_lic

# B_TYP = gp.GRB.BINARY
# I_TYP = gp.GRB.INTEGER
# C_TYP = gp.GRB.CONTINUOUS


# def model_with_one_var_each_type():
#     m = gp.Model()
#     b = m.addVar(vtype=B_TYP, name="b")
#     i = m.addVar(vtype=I_TYP, name="I")
#     c = m.addVar(vtype=C_TYP, name="C")
#     m.addConstr(b + i + c <= 5)
#     m.setObjective(b + i + c, gp.GRB.MAXIMIZE)
#     m.update()
#     return m


# # def model_with_two_var_each_type():
# #     m = gp.Model()
# #     b1 = m.addVar(vtype=B_TYP, name="b1")
# #     b2 = m.addVar(vtype=B_TYP, name="b2")

# #     i1 = m.addVar(vtype=I_TYP, name="i1")
# #     i2 = m.addVar(vtype=I_TYP, name="i2")

# #     c1 = m.addVar(vtype=C_TYP, name="c1")
# #     c2 = m.addVar(vtype=C_TYP, name="c2")
# #     m.addConstr(b1 + i1 + c1 <= 5)
# #     m.addConstr(b2 + i2 + c2 <= 5)
# #     m.addConstr(b1 + i1 + c1 + b2 + i2 + c2 >= 5)
# #     m.setObjective(b1 + i1 + c1, gp.GRB.MAXIMIZE)
# #     m.update()
# #     return m


# # m = model_with_one_var_each_type()
# # # m = model_with_two_var_each_type()
# # info = ModelInfo.from_model(m)

# # m = with_lic(m)
# # m.optimize()
# # vals = [x.x for x in m.getVars()]
# # sols = [[m.ObjVal] + vals]
# # m.dispose()

# # info.var_info.sols = np.array(sols)


In [11]:
from temp.data.graph import get_bipartite_graph, add_label
from temp.data.dataset import create_data_object

In [12]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info)
train_d = ModelGraphDataset(TRAIN_DIR, augment=augment_info)
valid_d = ModelGraphDataset(VALID_DIR)

# for i in range(len(pre_train_d)):
#     print(i)
#     data = pre_train_d[i][1]

data = pre_train_d[0][1]

Processing...


(model, solution) => info: 100%|██████████| 50/50 [00:14<00:00,  3.50it/s]
Done!
Processing...
(model, solution) => info: 100%|██████████| 2960/2960 [00:59<00:00, 49.98it/s]
Done!


In [13]:
# from pyinstrument import Profiler

# # Create a profiler with a smaller sampling interval for higher resolution
# profiler = Profiler()  # Samples every 0.5ms

# # Start profiling
# with profiler:
#     for i in range(5):
#         data = pre_train_d[i][1]

# html_report = profiler.output_html()
# with open("profile_report.html", "w") as f:
#     f.write(html_report)

In [14]:
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [15]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 16
CFG["num_layers"] = 8
CFG["hidden"] = 64

In [16]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [17]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [18]:
scheduler.total_steps = 10000

In [19]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=2, shuffle=True, num_workers=8, prefetch_factor=2, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=2, shuffle=True, num_workers=8, prefetch_factor=2, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=2, shuffle=True, num_workers=8, generator=torch.Generator().manual_seed(0))

In [20]:
scheduler.total_steps = 10000000

In [21]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  0%|          | 0/25 [00:33<?, ?it/s]


>> Epoch 1 ----------------------------------------------------------------------------------------------------
Training... 0


100%|██████████| 1480/1480 [1:13:49<00:00,  2.99s/it]


train_loss                0.297535
train_acc                 0.956952
train_f1                  0.240606
train_precision           0.358058
train_recall              0.181176
train_evidence_succ       7.435205
train_evidence_fail       3.884441
train_uncertainty_succ    0.270075
train_uncertainty_fail    0.382995
train_true_bias           0.037673
train_pred_bias           0.019056
train_soft_pred_bias      0.042310
train_bias_error          0.039260
train_lr                  0.000017
dtype: float32
Validating...


100%|██████████| 175/175 [02:09<00:00,  1.36it/s]


val_loss                 0.134528
val_acc                  0.983905
val_f1                   0.556206
val_precision            0.949897
val_recall               0.393230
val_evidence_succ       13.007076
val_evidence_fail        3.917455
val_uncertainty_succ     0.155713
val_uncertainty_fail     0.408878
val_true_bias            0.025657
val_pred_bias            0.010616
val_soft_pred_bias       0.014142
val_bias_error           0.015040
val_lr                   0.000017
dtype: float32
>> Epoch 2 ----------------------------------------------------------------------------------------------------
Training... 1480


 99%|█████████▉| 1472/1480 [1:04:29<00:21,  2.63s/it]


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/share/c3/v8/conda/Miniconda3-py38_4.9.2-MacOSX-x86_64/envs/gnn4co/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/share/c3/v8/conda/Miniconda3-py38_4.9.2-MacOSX-x86_64/envs/gnn4co/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/share/c3/v8/conda/Miniconda3-py38_4.9.2-MacOSX-x86_64/envs/gnn4co/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/share/c3/v8/conda/Miniconda3-py38_4.9.2-MacOSX-x86_64/envs/gnn4co/lib/python3.10/site-packages/torch_geometric/data/dataset.py", line 291, in __getitem__
    data = self.get(self.indices()[idx])
  F

In [22]:
assert 1 == 2

AssertionError: 

In [22]:
from temp.data.dataset import info_to_data

In [23]:
to_dispose = []

In [24]:
import gurobipy as gp

m = gp.read("hard.lp")


Restricted license - for non-production use only - expires 2026-11-23
Read LP format model from file hard.lp
Reading time = 0.14 seconds
: 52256 rows, 52224 columns, 136704 nonzeros


In [40]:
gb_m = with_lic(m)
gb_m.setParam("TimeLimit", 150)
gb_m.setParam("NoRelHeurTime", 60)
gb_m.optimize()
to_dispose.append(gb_m)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: c5236a47-c775-4ba7-9062-0853e53fabf5
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection
Set parameter TimeLimit to value 150
Set parameter NoRelHeurTime to value 60
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  150
NoRelHeurTime  60
CSIdleTimeout  1800

Optimize a model with 52256 rows, 52224 columns and 136704 nonzeros
Model fingerprint: 0x68be263f
Variable types: 32768 continuous, 19456 integer (16384 binary)
Coefficient statisti

In [25]:
info = ModelInfo.from_model(m)
data = info_to_data(info)

In [26]:
with torch.no_grad():
    logits = model(data)

In [27]:
len(logits)

52224

In [28]:
import gurobipy as gp

def get_binary_mask(model):
    vs = model.getVars()
    ts = model.getAttr("vType", vs)
    mask = [t == gp.GRB.BINARY for t in ts]
    mask = torch.as_tensor(mask, dtype=bool)
    return mask

def get_binary_indices(model):
    vs = model.getVars()
    ts = model.getAttr("vType", vs)
    idxs = [i for i, t in enumerate(ts) if t == gp.GRB.BINARY]
    idxs = torch.as_tensor(idxs, dtype=int)
    return idxs

In [29]:
binary_mask = get_binary_mask(m)

In [30]:
from temp.solver import upr

In [31]:
probs, preds = upr.get_predictions(logits, binary_mask)

In [32]:
preds

array([4.77789330e-14, 6.28900654e-21, 1.05960095e-14, ...,
       5.17547586e-17, 4.27185549e-17, 2.53556976e-18], dtype=float32)

In [33]:
unc = upr.get_uncertainty(logits)
unc = unc[binary_mask]

In [34]:
threshold = upr.get_threshold(unc)

In [35]:
import json
from tempfile import NamedTemporaryFile

import gurobipy as gp


def fix_var(inst, idxs, vals):
    assert len(idxs) == len(vals)
    bounds = {}
    vs = inst.getVars()
    for idx, val in zip(idxs, vals):
        v = vs[idx]
        bounds[idx] = (v.lb, v.ub)
        v.setAttr("lb", val)
        v.setAttr("ub", val)
    inst.update()
    return bounds


def unfix_var(inst, idxs, bounds):
    assert len(idxs) == len(bounds)
    vs = inst.getVars()
    print(idxs, bounds)
    for i, (lb, ub) in zip(idxs, bounds):
        vs[i].setAttr("lb", lb)
        vs[i].setAttr("ub", ub)


def get_iis_vars(inst):
    try:
        inst.computeIIS()
    except Exception as e:
        print(e)
        if "Cannot compute IIS on a feasible model" in str(e):
            return set()
        raise e

    with NamedTemporaryFile(suffix=".ilp", mode="w+") as f:
        inst.write(f.name)
        f.seek(0)
        return set(f.read().split())


def set_starts(inst, starts):
    vs = inst.getVars()
    for i, s in starts.items():
        vs[i].setAttr("lb", s)


def solve_inst(inst):
    vs = inst.getVars()
    inst.optimize()
    return inst.getAttr("X", vs)


def repair(inst, fixed: set, bounds: dict):
    old_iis_method = getattr(inst, "IISMethod", -1)
    # inst.setParam("IISMethod", 0)

    vs = inst.getVars()
    ns = inst.getAttr("varName", vs)
    name_to_idx = {n: i for i, n in enumerate(ns)}

    freed = set()
    while iis_var_names := get_iis_vars(inst):
        for n in iis_var_names:
            if n not in name_to_idx:
                continue
            
            var_idx = name_to_idx[n]
            if var_idx not in fixed:
                continue

            if var_idx in freed:
                continue
            print(n, var_idx)
            if var_idx not in bounds:
                continue
            
            lb, ub = bounds[var_idx]
            vs[var_idx].lb = lb
            vs[var_idx].ub = ub
            freed.add(var_idx)

    inst.setParam("IISMethod", old_iis_method)
    return freed


def set_hints(inst, hints):
    vs = inst.getVars()
    for i, (val, pri) in hints:
        if 0 <= pri <= 1:
            pri = int(pri * 100)
        vs[i].setAttr("VarHintVal", val)
        vs[i].setAttr("VarHintPri", pri)


def with_lic(m, path="gb.lic"):
    with open(path) as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)


In [36]:
print(m.getVarByName("y[5,15]").ub)
print(m.getVarByName("y[5,15]").lb)

1.0
0.0


In [37]:
import numpy as np
import torch
import torch.nn.functional as F

# from temp.solver.utils import fix_var, repair, set_starts, unfix_var, solve_inst

EVIDENCE_FUNCS = {
    "softplus": (lambda y: F.softplus(y)),
    "relu": (lambda y: F.relu(y)),
    "exp": (lambda y: torch.exp(torch.clamp(y, -10, 10))),
}


def get_predictions(logits, binary_mask):
    probs = torch.softmax(logits, axis=1)
    preds = probs[:, 1]
    probs = _to_numpy(probs)
    preds = _to_numpy(preds).squeeze()
    preds[binary_mask] = preds[binary_mask].round()
    return probs, preds


def get_uncertainty(logits, evidence_func_name: str = "softplus"):
    evidence = EVIDENCE_FUNCS[evidence_func_name](logits)
    alpha = evidence + 1
    uncertainty = logits.shape[1] / torch.sum(alpha, dim=1, keepdim=True)
    return uncertainty


def get_threshold(uncertainty: torch.Tensor, r_min: float = 0.4, r_max: float = 0.55):
    q = (r_min + r_max) / 2
    threshold = torch.quantile(uncertainty, q)
    r = (uncertainty <= threshold).float().mean()

    if r > r_max:
        threshold = torch.quantile(uncertainty, r_max)
        return threshold

    if r < r_min:
        threshold = torch.quantile(uncertainty, r_min)
        return threshold

    return threshold


def get_confident_idx(indices, uncertainty, threshold):
    confident_mask = uncertainty <= threshold
    confident_idx = indices[confident_mask]
    return confident_idx.sort()[0]


def solve(inst, prediction, uncertainty, indices, max_iter):
    threshold = get_threshold(uncertainty)
    conf_idxs = get_confident_idx(indices, uncertainty, threshold)
    conf_vals = prediction[conf_idxs]

    conf_idxs = conf_idxs.tolist()
    bounds = fix_var(inst, conf_idxs, conf_vals)

    print(len(bounds), len(prediction))

    min_q = sum(uncertainty <= threshold) / len(uncertainty)
    max_q = 1.0
    dq = (max_q - min_q) / (max_iter - 1)

    fixed = set(conf_idxs)
    freed = set(repair(inst, fixed, bounds))
    for i in range(1, max_iter):
        sol = solve_inst(inst)
        q = max_q - dq * i
        threshold = np.quantile(uncertainty, q)
        conf_idxs = get_confident_idx(indices, uncertainty, threshold)
        to_unfix = list(fixed - set(conf_idxs.tolist()))
        to_unfix = [i for i in to_unfix if i not in freed]
        print(len(to_unfix))
        unfix_var(inst, to_unfix, [bounds[i] for i in to_unfix])
        starts = {i: sol[i] for i in to_unfix}
        starts.update({i: sol[i] for i in freed})
        set_starts(inst, starts)
    unfix_var(inst, bounds.keys(), bounds.values())
    inst.setParam("TimeLimit", 100)
    sol = solve_inst(inst)
    return sol


def _to_numpy(tensor_obj):
    return tensor_obj.cpu().detach().numpy()


In [38]:
for d in to_dispose:
    d.dispose()

In [39]:
indices = get_binary_indices(m)
unc = unc.squeeze()
to_solve = with_lic(m)
to_dispose.append(to_solve)
to_solve.setParam("TimeLimit", 100)
solve(to_solve, preds, unc, indices, 5)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Waiting for cloud server to start (pool 831775-C3Dev)...
Starting...
Starting...
Starting...
Compute Server job ID: 81356808-0ddb-4da5-8829-cc3853cb6b51
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection
Set parameter TimeLimit to value 100
7782 52224
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  100
CSIdleTimeout  1800

IIS computation: initial model status unknown, solving to determine model status
IIS runtime: 0.00 seconds (0.00 work units)
Cannot com

[1309984920.0,
 654992460.0,
 218330820.0,
 109165410.0,
 36388470.0,
 18194235.0,
 18194232.0,
 18194232.0,
 9097116.0,
 3032372.0,
 1010790.6666666666,
 336930.22222222225,
 168465.11111111112,
 168465.11111111112,
 56154.99948937437,
 12998.84247439222,
 12998.84247439222,
 3610.78957622006,
 1504.495656758359,
 1253.746380631965,
 520.0,
 216.0,
 151.15576541183992,
 50.38525513727998,
 20.9938563072,
 17.494880256,
 14.579066880000001,
 10.760739840000005,
 10.124352,
 2.81232,
 0.7812,
 0.3255,
 863261406.0,
 431630703.0,
 143876901.0,
 71938450.0,
 23979483.333333332,
 11989741.666666666,
 11989744.0,
 11989744.0,
 5994870.0,
 1998290.0,
 666096.6666666666,
 222032.22222222222,
 111016.11111111111,
 111016.11111111111,
 37005.36832709923,
 8566.057483124823,
 8566.057483124823,
 2379.460411979117,
 991.4418383246323,
 826.2015319371936,
 340.0,
 138.0,
 99.60955969535998,
 33.20318656511999,
 13.8346610688,
 11.528884223999997,
 9.607403519999998,
 8.006169599999998,
 6.67180800